In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import warnings
import matplotlib. as plt
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../Data/Train_TEC_mar17.csv")
df.describe()

,Date,Month,Year,Time,LT,TEC
count,14400.000000,14400.0,14400.0,14400.000000,14400.000000,14400.000000
mean,10.500000,3.0,2017.0,11.999714,17.294738,17.016025
std,5.766482,0.0,0.0,6.928431,6.928072,10.732619
min,1.000000,3.0,2017.0,0.016389,5.310988,0.285998
25%,5.750000,3.0,2017.0,6.008056,11.303727,7.154763
50%,10.500000,3.0,2017.0,11.999583,17.294961,16.224494
75%,15.250000,3.0,2017.0,17.991319,23.285830,27.533690
max,20.000000,3.0,2017.0,23.983056,29.278233,42.039814


In [3]:
df.drop(["Date","Month","Year","Time","LT"], axis=1, inplace=True)
df.head()

,TEC
0,2.067562
1,2.082707
2,2.004272
3,1.736569
4,1.758503


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TEC     14400 non-null  float64
dtypes: float64(1)
memory usage: 112.6 KB


In [6]:
df.TEC = df.TEC.astype('float32')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TEC     14400 non-null  float32
dtypes: float32(1)
memory usage: 56.4 KB


In [8]:
train_data = df[:-1440]
test_data = df[-1440:]

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [10]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 12
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(200, activation='tanh', input_shape=(n_input, n_features)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               161600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________


In [11]:
lstm_model.fit_generator(generator,epochs=20)

Epoch 1/20
12948/12948 [==============================] - 61s 5ms/step - loss: 3.0320e-04
Epoch 2/20
12948/12948 [==============================] - 61s 5ms/step - loss: 8.4532e-05
Epoch 3/20
12948/12948 [==============================] - 62s 5ms/step - loss: 7.1320e-05
Epoch 4/20
12948/12948 [==============================] - 61s 5ms/step - loss: 6.4532e-05
Epoch 5/20
12948/12948 [==============================] - 60s 5ms/step - loss: 6.0942e-05
Epoch 6/20
12948/12948 [==============================] - 60s 5ms/step - loss: 5.9501e-05
Epoch 7/20
12948/12948 [==============================] - 62s 5ms/step - loss: 5.6776e-05
Epoch 8/20
12948/12948 [==============================] - 63s 5ms/step - loss: 5.6151e-05
Epoch 9/20
12948/12948 [==============================] - 61s 5ms/step - loss: 5.5096e-05
Epoch 10/20
12948/12948 [==============================] - 63s 5ms/step - loss: 5.3045e-05
Epoch 11/20
12948/12948 [==============================] - 61s 5ms/step - loss: 5.3386e-05
Epoch 12

In [12]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize=(12,4))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)),losses_lstm);

NameError: name 'plt' is not defined